In [1]:
from __future__ import absolute_import, division, print_function, unicode_literals
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers, models
from tensorflow.keras.mixed_precision import experimental as mixed_precision

from tensorflow.keras.callbacks import Callback

import IPython.display as display
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import pathlib

import tensorflow.keras.applications

from tensorflow.keras.applications import DenseNet121,DenseNet169,  \
                                          DenseNet201,InceptionResNetV2,  \
                                          InceptionV3,MobileNet,MobileNetV2,  \
                                          NASNetLarge,NASNetMobile,ResNet101,  \
                                          ResNet101V2,ResNet152,ResNet152V2,  \
                                          ResNet50,ResNet50V2,VGG16,VGG19,Xception  
AUTOTUNE = tf.data.experimental.AUTOTUNE

BATCH_SIZE = 16
IMG_HEIGHT = 512
IMG_WIDTH = 512
EPOCHS = 1
lr = 3e-3
CHANNELS =3
IMG_SHAPE = (IMG_HEIGHT, IMG_WIDTH, CHANNELS)

FILTER_SIZE = 3

lrs = [1e-4, 3e-3, 1e-2]
batch_sizes = [16,32,64]

In [2]:
!ls /opt/AIStorage/PLAYGROUND/images/512/filtered/data_filtered/train/data_filtered

negative  positive


In [3]:
train_data_dir = '/opt/AIStorage/PLAYGROUND/images/512/filtered/data_filtered/train/data_filtered/'

data_dir_path = pathlib.Path(train_data_dir)

CLASS_NAMES = np.array([item.name for item in data_dir_path.glob('*')])

CLASS_NAMES

array(['positive', 'negative'], dtype='<U8')

In [4]:
import glob

image_list = glob.glob(train_data_dir + '**/*.png', recursive=True)
DATASET_SIZE = len(image_list)
DATASET_SIZE

8827

In [5]:
image_count = int(0.8 * DATASET_SIZE)
val_image_count = int(0.2 * DATASET_SIZE)

In [6]:
image_count

7061

In [7]:
val_image_count

1765

In [8]:


full_list_ds = tf.data.Dataset.from_tensor_slices(image_list)

full_list_ds = full_list_ds.shuffle(buffer_size=10000)

train_list_ds = full_list_ds.take(image_count)

val_list_ds = full_list_ds.skip(image_count)


In [9]:
def get_label(file_path):
    # convert the path to a list of path components
    parts = tf.strings.split(file_path, os.path.sep)
    # The second to last is the class-directory
    label = -1
    if parts[-2] == 'negative':
        label = tf.constant([1.0, 0.0])
    else:
        label = tf.constant([0.0, 1.0])
    return label

def decode_img(img):
    # convert the compressed string to a 3D uint8 tensor
    img = tf.image.decode_png(img, channels=CHANNELS)
    # Use `convert_image_dtype` to convert to floats in the [0,1] range.
    img = tf.image.convert_image_dtype(img, tf.float32)
    # resize the image to the desired size.
    return tf.image.resize(img, [IMG_WIDTH, IMG_HEIGHT])

def process_path(file_path):
    label = get_label(file_path)
    # load the raw data from the file as a string
    img = tf.io.read_file(file_path)
    img = decode_img(img)
    return img, label

In [10]:
# Set `num_parallel_calls` so multiple images are loaded/processed in parallel.
labeled_ds = train_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

val_labeled_ds = val_list_ds.map(process_path, num_parallel_calls=AUTOTUNE)

In [11]:
for image, label in labeled_ds.take(5):
    print("Image shape: ", image.numpy().shape)
    print("Label: ", label.numpy())

Image shape:  (512, 512, 3)
Label:  [0. 1.]
Image shape:  (512, 512, 3)
Label:  [0. 1.]
Image shape:  (512, 512, 3)
Label:  [0. 1.]
Image shape:  (512, 512, 3)
Label:  [1. 0.]
Image shape:  (512, 512, 3)
Label:  [1. 0.]


In [12]:
STEPS_PER_EPOCH = np.ceil(image_count/BATCH_SIZE)
VAL_STEPS_PER_EPOCH = np.ceil(val_image_count/BATCH_SIZE)

def prepare_for_training(ds, shuffle=True, cache=True, shuffle_buffer_size=11000):
    # This is a small dataset, only load it once, and keep it in memory.
    # use `.cache(filename)` to cache preprocessing work for datasets that don't
    # fit in memory.
    if cache:
        if isinstance(cache, str):
            ds = ds.cache(cache)
        else:
            ds = ds.cache()

    if shuffle:
        ds = ds.shuffle(buffer_size=shuffle_buffer_size, reshuffle_each_iteration=True)

    # Repeat forever
    ds = ds.repeat()

    ds = ds.batch(BATCH_SIZE)

    # `prefetch` lets the dataset fetch batches in the background while the model
    # is training.
    ds = ds.prefetch(buffer_size=AUTOTUNE)

    return ds

train_ds = prepare_for_training(labeled_ds)

valid_ds = prepare_for_training(val_labeled_ds, shuffle=False)

steps = np.ceil(image_count / BATCH_SIZE) * EPOCHS

optimizer = tf.keras.optimizers.Adam(lr=lr)

model = models.Sequential()
model.add(layers.Conv2D(32, (3, 3), activation='relu', input_shape=(512, 512, 3)))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu'))
model.add(layers.MaxPooling2D((2, 2)))
model.add(layers.Conv2D(64, (3, 3), activation='relu', name="target_layer"))

model.add(layers.Flatten())
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(2))


model.compile(optimizer=optimizer,loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),metrics=['acc'])
    
    
history = model.fit(train_ds,
                epochs=EPOCHS,
                validation_data=valid_ds, 
                steps_per_epoch=STEPS_PER_EPOCH, 
                validation_steps=VAL_STEPS_PER_EPOCH,
                callbacks=[])


Train for 442.0 steps, validate for 111.0 steps
442/442 [==============================] - 50s 113ms/step - loss: 0.8463 - acc: 0.5214 - val_loss: 0.6800 - val_acc: 0.5152


In [13]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 510, 510, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 255, 255, 32)      0         
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 253, 253, 64)      18496     
_________________________________________________________________
max_pooling2d_1 (MaxPooling2 (None, 126, 126, 64)      0         
_________________________________________________________________
target_layer (Conv2D)        (None, 124, 124, 64)      36928     
_________________________________________________________________
flatten (Flatten)            (None, 984064)            0         
_________________________________________________________________
dense (Dense)                (None, 64)                6

In [2]:
import tensorflow as tf

from tf_explain.core.grad_cam import GradCAM

In [3]:
it = iter(val_list_ds)

NameError: name 'val_list_ds' is not defined

In [ ]:
print(next(it).numpy())

In [ ]:
IMAGE_PATH = '/opt/AIStorage/PLAYGROUND/images/512/filtered/data_filtered/train/data_filtered/negative/2.25.303281520734538440475824697948406608516.png'

In [ ]:
img = tf.keras.preprocessing.image.load_img(IMAGE_PATH, target_size=(512, 512))
img = tf.keras.preprocessing.image.img_to_array(img)

In [ ]:
img = img/255.0

In [ ]:
data = ([img], None)

negative_index = 0
explainer = GradCAM()

In [ ]:

    
grid = explainer.explain(
    data, model, class_index=0, layer_name="target_layer"
)
    
explainer.save(grid, ".", "grad_cam2.png")